# Explore and Model StatFox Matchup Data
`mlb_bet_notebooks/model_statfox_matchups.ipynb`
- Explore features
- Convert historical moneylines to break-even probabilities
- Model pre-computed features with RF and maybe PCA
- Compare model predictions to historical moneylines
    - Use break-even probabilities as alternative model and compare ROC
- Try VIF filter
- Try k-fold CV
- Try grid search model complexity
- Try to get player salary
    - Combine with addition, subtraction from statfox blobs
- Try fix Opening Line feature 
    - Try openline probability as feature

Jonathan Sims 2020-02-24

In [1]:
### Set datetime stamp in YYYYMMDD for all file outputs

from datetime import date as dt
YMD = dt.today().isoformat().replace('-','') + '.'

In [2]:
import math
import boto3
import pandas as pd 
import pickle
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import roc_auc_score, make_scorer
import numpy as np
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
import random
from sklearn.model_selection import train_test_split, GridSearchCV, LeaveOneOut

### Import model data

In [3]:
s3 = boto3.client('s3')

In [4]:
df_feat_fill = pd.read_csv('s3://scrapes-rawhtml-dev/statfox/20200315.statfox_features.tsv.gz', sep='\t', index_col=0)
df_targ = pd.read_csv('s3://scrapes-rawhtml-dev/statfox/20200315.statfox_target.tsv.gz', sep='\t', index_col=0, header=None, squeeze=True)
df_targ_wt = pd.read_csv('s3://scrapes-rawhtml-dev/statfox/20200315.statfox_target_weight.tsv.gz', sep='\t', index_col=0, header=None, squeeze=True)
df_lateline_prob = pd.read_csv('s3://scrapes-rawhtml-dev/statfox/20200315.statfox_lateline_prob.tsv.gz', sep='\t', index_col=0, header=None, squeeze=True)

In [5]:
df_targ.shape

(17573,)

In [6]:
df_feat_fill.shape

(17573, 1128)

In [7]:
df_targ_wt.head()

0
0    1.769231
1    1.606061
2    2.050000
3    1.833333
4    1.769231
Name: 1, dtype: float64

In [8]:
df_targ_wt.sum()

34455.36163974437

In [9]:
### Make sure matchidx column exists

[col for col in df_feat_fill.columns if 'match' in col]

['matchidx']

In [10]:
### Partial header names to looks for unknown/erroneous columns

f = lambda x: x[2:13]
colstrip = pd.Series(df_feat_fill.columns).map(f)
[col for col in colstrip.drop_duplicates() if '_h_' not in col and '_v_' not in col]

['named: 0.1',
 'Bullpen_BB_',
 'Bullpen_BSV',
 'Bullpen_ERA',
 'Bullpen_ER_',
 'Bullpen_HR_',
 'Bullpen_H_A',
 'Bullpen_H_H',
 'Bullpen_IP_',
 'Bullpen_L_A',
 'Bullpen_L_H',
 'Bullpen_R_A',
 'Bullpen_R_H',
 'Bullpen_SO_',
 'Bullpen_SV_',
 'Bullpen_WHI',
 'Bullpen_W_A',
 'Bullpen_W_H',
 'HitField_Te',
 'Overall_Opp',
 'Overall_Tea',
 'Bullpen_H_R',
 'Bullpen_L_R',
 'Bullpen_R_R',
 'Bullpen_W_R',
 'tchidx',
 'Bullpen_Pct',
 '_Opening_Li',
 'nth',
 'ar',
 '_Latest_Tot',
 '_Opening_To']

### Make custom score function

In [24]:
def test_score(y, y_pred, wt):
    y_diff = (-1 * abs(y - y_pred) + 1 ) * wt - 1
    y_diff_sum = y_diff.sum() / len(y)
    return y_diff_sum

In [25]:
sum_score = make_scorer(test_score, wt=pd.Series(df_targ_wt.values))

### Calculate close moneyline ROC AUC

In [26]:
df_targ_dropna = df_targ[df_lateline_prob.isna() == False]
df_lateline_prob_dropna = df_lateline_prob[df_lateline_prob.isna() == False]
df_targ_wt_dropna = df_targ_wt[df_lateline_prob.isna() == False]

In [27]:
roc_auc_score(df_targ_dropna, df_lateline_prob_dropna, sample_weight=df_targ_wt_dropna)

0.5071739747513021

In [15]:
df_targ.shape

(17573,)

In [16]:
df_feat_fill.shape

(17573, 1128)

In [17]:
df_targ_wt.shape

(17573,)

In [18]:
df_targ_wt.values

array([1.76923077, 1.60606061, 2.05      , ..., 1.625     , 2.15      ,
       1.60606061])

### RF with AUC and no PCA

In [19]:
df_feat_fill.shape

(17573, 1128)

## Tune Parameters

### Big gridsearch with n_jobs=1 and no sample weight

In [ ]:
%%time 


### Define len for subset for development

_dev_len = 16000


### Define feature and target data

# X = df_feat_fill[:_dev_len].to_numpy()
# y = df_targ[:_dev_len].to_numpy()
X = df_feat_fill[:_dev_len]
y = df_targ[:_dev_len]


### Define custom profit score function

prof_score = make_scorer(test_score, wt=df_targ_wt)


### Save number of splits for leave-one-out CV

loo = LeaveOneOut()
splits = loo.split(X)


### Grid of hyperparams to search

max_depth_par = range(2,3)
# min_samples_split_par = range(24,32)
min_samples_leaf_par = range(6,7)
criterion_par = ['entropy']
class_weight_par = ['balanced_subsample']
oob_score_par = [False]

parameters = {'max_depth':max_depth_par, 
#               'min_samples_split':min_samples_split_par, 
              'min_samples_leaf':min_samples_leaf_par, 
              'criterion':criterion_par, 
              'class_weight':class_weight_par, 
              'oob_score':oob_score_par}
#                 }

rfc = RandomForestClassifier(n_estimators=10, n_jobs=1)
clf = GridSearchCV(rfc, parameters, n_jobs=-1, scoring=prof_score, cv=splits)

fit = clf.fit(X, y)

par = clf.best_params_
# fea = clf.feature
scr = clf.score(X, y)
sco = clf.score(df_feat_fill[_dev_len:], df_targ[_dev_len:])

pkl = [fit, par, scr, sco]
# with open(YMD+'GridSearchCV.RandomForest.pkl', 'wb') as wb:
#     pickle.dump(pkl, wb)

In [ ]:
for x in pkl:
    print(x)

In [ ]:
clf.best_params_

In [30]:
clf.score(df_feat_fill[_dev_len:], df_targ[_dev_len:])

NameError: name 'clf' is not defined

### Big gridsearch with n_jobs=2 and sample weighting

In [31]:
%%time 


### Define len for subset for development

_dev_len = 16000


### Define feature and target data

# X = df_feat_fill[:_dev_len].to_numpy()
# y = df_targ[:_dev_len].to_numpy()
X = df_feat_fill[:_dev_len]
y = df_targ[:_dev_len]


### Define custom profit score function

prof_score = make_scorer(test_score, wt=df_targ_wt)


### Save number of splits for leave-one-out CV

loo = LeaveOneOut()
splits = loo.split(X)


### Grid of hyperparams to search
max_depth_par = range(1,5)
# min_samples_split_par = range(15,30)
# min_samples_leaf_par = range(3,15)
# criterion_par = ['gini', 'entropy']
# class_weight_par = ['balanced', 'balanced_subsample']
oob_score_par = [False, True]

parameters = {'max_depth':max_depth_par, 
#               'min_samples_split':min_samples_split_par, 
#               'min_samples_leaf':min_samples_leaf_par, 
#               'criterion':criterion_par, 
#               'class_weight':class_weight_par, 
              'oob_score':oob_score_par}
#                 }

rfc = RandomForestClassifier(n_estimators=100, n_jobs=2)
clf = GridSearchCV(rfc, parameters, n_jobs=-1, scoring=prof_score, cv=10)


### Write estimator and results to pickles

fit = clf.fit(X, y, sample_weight=df_targ_wt[:_dev_len])
par = clf.best_params_
scr = clf.score(X, y)
sco = clf.score(df_feat_fill[_dev_len:], df_targ[_dev_len:])
scw = clf.score(df_feat_fill[_dev_len:], df_targ[_dev_len:], sample_weight=df_targ_wt[_dev_len:])

pkl = [fit, par, scr, sco, scw]
with open(YMD+'GridSearchCV.RandomForest.Weight.pkl', 'wb') as wb:
    pickle.dump(pkl, wb)

TypeError: score() got an unexpected keyword argument 'sample_weight'

In [ ]:
clf.best_params_

In [ ]:
clf.score(df_feat_fill[_dev_len:], df_targ[_dev_len:])

# `20200318`

In [30]:
X_val = df_feat_fill[_dev_len:].to_numpy()
y_val = df_targ[_dev_len:].to_numpy()

clf_pred = clf.predict(X_val)
roc_auc_score(y_val, clf_pred)

0.546827326160997

In [ ]:
df_feat_fill.shape

In [ ]:
X.shape

In [ ]:
for cnt in range(1,100,10):
#     X_train, X_test, Y_train, Y_test = train_test_split(df_feat_fill, df_targ, test_size=0.2, random_state=cnt)

    ### Standarize data

    min_max_scaler = preprocessing.MinMaxScaler()
    np_scaled = min_max_scaler.fit_transform(df_feat_fill)
    df_feat_fill_st = pd.DataFrame(np_scaled)

    df_feat_fill_train, df_feat_fill_test, df_targ_train, df_targ_test = train_test_split(df_feat_fill_st, df_targ, test_size=0.2)

    clf = AdaBoostClassifier()
    df_fit = clf.fit(df_feat_fill_train, df_targ_train)
    df_pred = df_fit.predict(df_feat_fill_test)
    
    print(roc_auc_score(df_targ_test, df_pred))
    

## Test out PCA

### Standardize Features

In [75]:
def RunPCA(X,n):
    """Takes an input data set X and returns n principal components
    """
    # Create a scaler object
    sc = StandardScaler()
    
    # Fit the scaler to the features and transform
    X_std = sc.fit_transform(X)

    # Create a pca object with the 2 components as a parameter
    pca = decomposition.PCA(n_components=n)

    # Fit the PCA and transform the data
    X_std_pca = pca.fit_transform(X_std)
    
    return X_std_pca

In [76]:
df_feat_fill_train = df_feat_fill.iloc[:nsplit]
df_targ_train = df_targ.iloc[:nsplit]
df_feat_fill_test = df_feat_fill.iloc[nsplit:]
df_targ_test = df_targ.iloc[nsplit:]

### Practice with PCA

transform df with fit on train

### Tune n_components param

In [94]:
scores = dict()

for cnt in range(1,100,5):

    ### Create a pca object with the 2 components as a parameter
    
    pca = decomposition.PCA(n_components=50)

    df_feat_fill_train, df_feat_fill_test, df_targ_train, df_targ_test = train_test_split(df_feat_fill, df_targ, test_size=0.1, random_state=1)

    X1 = df_feat_fill_train
    X2 = df_feat_fill_test

    ### Create a scaler object
    
    sc = StandardScaler()

    ### Fit the scaler to the features and transform
    
    X1_std = sc.fit_transform(X1)
    X2_std = sc.fit(X1).transform(X2)

    ### Fit the PCA and transform the data
    
    X1_std_pca = pca.fit_transform(X1_std)
    X2_std_pca = pca.fit(X1_std).transform(X2_std)

    std_pca_train = X1_std_pca
    std_pca_df = X2_std_pca

    clf = AdaBoostClassifier(n_estimators=100, random_state=1)
    df_fit = clf.fit(std_pca_train, df_targ_train)
    df_pred = df_fit.predict(std_pca_df)
    score = roc_auc_score(df_targ_test, df_pred)
    
    ### Append score to dict
    
    scores[cnt] = score
    
    ### Print for OCD
    print(cnt,' ',score)

1   0.5406012163200662
6   0.5254810861839616
11   0.5257931631733549
16   0.5242600849629604
21   0.5194033868155274
26   0.5261052401627482
31   0.5394582343464133
36   0.5252879885467745
41   0.5288456662258579
46   0.530548436299235
51   0.512311437231565
56   0.5231171029893075
61   0.5362496927992135
66   0.5289373388414922
71   0.5312876686678604
76   0.5225846216261553
81   0.5338584028679876
86   0.5248198730626845
91   0.5148587656574877
96   0.531898169278361


In [ ]:
RandomForestClassifier?

In [ ]:
pd.DataFrame([clf.feature_importances_, df_feat_fill_train.columns]).transpose().sort_values

In [ ]:
pd.DataFrame([clf.feature_importances_, df_feat_fill_train.columns]).transpose.sort

In [ ]:
df_feat_fill_desc = df_feat_fill_train.describe().loc[['mean', 'std']]

In [ ]:
df_feat_fill_desc

In [ ]:
for x in df_feat_fill_desc.columns:
    print(df_feat_fill_desc[x])

In [ ]:
scores = dict()
for cnt in range(4):
    scores[cnt] = cnt*4

In [ ]:
scores